# A little non-economic research


## Existing quiz tests

https://www.linkedin.com/skill-assessments/hub/quizzes/ 

![](../docs/li.png)

https://www.w3schools.com/quiztest/quiztest.asp?qtest=PANDAS


![](../docs/w3.png)


## Code interview

Datasets:

1. Binance Open Data: [spot candles](https://github.com/binance/binance-public-data/#klines)
2. OpenFIGI: [search API](https://www.openfigi.com/search).

### Binance Open Data

Downloading candles for `BTC/USDT` and `BTC/UDSC` using `bash` or `powershell`:

In [2]:
#!/bin/sh

# create dir for data
!mkdir ../data/binance

# download data using GET request
!wget -N -P ../data/binance https://data.binance.vision/data/spot/daily/klines/BTCUSDT/1m/BTCUSDT-1m-2022-06-21.zip
!wget -N -P../data/binance https://data.binance.vision/data/spot/daily/klines/BTCUSDC/1m/BTCUSDC-1m-2022-06-21.zip

# unzip
!unzip -o -d ../data/binance ../data/binance/BTCUSDT-1m-2022-06-21.zip 
!unzip -o -d ../data/binance ../data/binance/BTCUSDC-1m-2022-06-21.zip

mkdir: cannot create directory ‘../data/binance’: File exists
--2022-06-23 12:48:13--  https://data.binance.vision/data/spot/daily/klines/BTCUSDT/1m/BTCUSDT-1m-2022-06-21.zip
Resolving data.binance.vision (data.binance.vision)... 13.224.2.90, 13.224.2.55, 13.224.2.128, ...
Connecting to data.binance.vision (data.binance.vision)|13.224.2.90|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘../data/binance/BTCUSDT-1m-2022-06-21.zip’ not modified on server. Omitting download.

--2022-06-23 12:48:14--  https://data.binance.vision/data/spot/daily/klines/BTCUSDC/1m/BTCUSDC-1m-2022-06-21.zip
Resolving data.binance.vision (data.binance.vision)... 13.224.2.90, 13.224.2.55, 13.224.2.128, ...
Connecting to data.binance.vision (data.binance.vision)|13.224.2.90|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘../data/binance/BTCUSDC-1m-2022-06-21.zip’ not modified on server. Omitting download.

Archive:  ../data/binance/BTCUSDT-1m-202

Import packages for data analysis

In [3]:
import numpy as np
import pandas as pd

import httpx

from datetime import datetime

Read data from CSV file to Pandas DataFrame:

In [4]:
def get_data(pair: str) -> pd.DataFrame:
    return pd.read_csv(f'../data/binance/{pair}-1m-2022-06-21.csv', header = None)

btcusdt_df = get_data('BTCUSDT')
btcusdt_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1655769600000,20573.90,20590.00,20552.17,20558.36,70.76925,1655769659999,1.455321e+06,1150,37.36821,7.683845e+05,0
1,1655769660000,20558.35,20611.21,20558.35,20606.70,118.06032,1655769719999,2.430514e+06,1402,61.25760,1.260950e+06,0
2,1655769720000,20606.69,20626.89,20552.40,20552.40,130.42894,1655769779999,2.686026e+06,1433,55.80573,1.149409e+06,0
3,1655769780000,20552.41,20585.69,20539.09,20578.89,103.56318,1655769839999,2.128819e+06,1301,64.57346,1.327338e+06,0
4,1655769840000,20578.89,20579.90,20537.57,20554.46,83.55509,1655769899999,1.717907e+06,1098,36.40944,7.485065e+05,0


Set names to columns with 1m candles:

In [5]:
def set_column_names(df: pd.DataFrame) -> pd.DataFrame:
    column_names_mapping = {
        0: 'Open_time',
        1: 'Open',
        2: 'High',
        3: 'Low',
        4: 'Close',
        5: 'Volume',
        6: 'Close_time',
        7: 'Quote_asset_volume',
        8: 'Number_of_trades',
        9: 'Taker_buy_base_asset_volume',
        10: 'Taker_buy_quote_asset_volume',
        11: 'Ignore'
        }
    return df.rename(columns=column_names_mapping)

btcusdt_df = set_column_names(btcusdt_df)
btcusdt_df.head()

,Open_time,Open,High,Low,Close,Volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,Ignore
0,1655769600000,20573.90,20590.00,20552.17,20558.36,70.76925,1655769659999,1.455321e+06,1150,37.36821,7.683845e+05,0
1,1655769660000,20558.35,20611.21,20558.35,20606.70,118.06032,1655769719999,2.430514e+06,1402,61.25760,1.260950e+06,0
2,1655769720000,20606.69,20626.89,20552.40,20552.40,130.42894,1655769779999,2.686026e+06,1433,55.80573,1.149409e+06,0
3,1655769780000,20552.41,20585.69,20539.09,20578.89,103.56318,1655769839999,2.128819e+06,1301,64.57346,1.327338e+06,0
4,1655769840000,20578.89,20579.90,20537.57,20554.46,83.55509,1655769899999,1.717907e+06,1098,36.40944,7.485065e+05,0


Convert timestamp to human-readable date and time format:

In [6]:
btcusdt_df['Open_time'] = btcusdt_df.iloc[:, 0].apply(lambda t: datetime.fromtimestamp(t/1000))
btcusdt_df['Close_time'] = btcusdt_df.iloc[:, 6].apply(lambda t: datetime.fromtimestamp(t/1000))

Find min and max time:

In [7]:
btcusdt_df[['Open_time', 'Close_time']].aggregate(func=[min, max, len])

,Open_time,Close_time
min,2022-06-21 00:00:00,2022-06-21 00:00:59.999000
max,2022-06-21 23:59:00,2022-06-21 23:59:59.999000
len,1440,1440


Calculate 1-hour `OHLCV` candles:

In [8]:
def calculate_ohclv(df: pd.DataFrame) -> pd.DataFrame:
    df['hour'] = df['Close_time'].apply(lambda t: t.hour)

    return (
        df
            .groupby(['hour'])
            .agg(
                {
                    'Open': 'first',
                    'High': max,
                    'Low': min,
                    'Close': 'last',
                    'Volume': sum,
                    'Close_time': max
                }
            )
            .reset_index()
            .drop(columns=['hour'])
        )

btcusdt_1h_df = calculate_ohclv(btcusdt_df)
btcusdt_1h_df

,Open,High,Low,Close,Volume,Close_time
0,20573.90,20705.74,20396.77,20672.30,4235.77392,2022-06-21 00:59:59.999
1,20672.30,20783.99,20504.89,20670.88,3892.91412,2022-06-21 01:59:59.999
2,20670.87,20699.93,20348.40,20433.49,2876.88928,2022-06-21 02:59:59.999
3,20433.49,20665.26,20365.00,20614.04,3306.77018,2022-06-21 03:59:59.999
4,20614.04,20740.72,20474.41,20656.17,2925.33542,2022-06-21 04:59:59.999
5,20656.17,21029.93,20621.14,20890.77,6516.65611,2022-06-21 05:59:59.999
6,20890.77,21202.00,20890.77,21192.08,6114.51662,2022-06-21 06:59:59.999
7,21192.08,21333.00,20952.46,21120.28,5433.08603,2022-06-21 07:59:59.999
8,21120.28,21500.01,21051.25,21356.32,6241.36801,2022-06-21 08:59:59.999
9,21356.32,21470.00,21166.94,21200.00,4961.57496,2022-06-21 09:59:59.999


Validate results:

In [9]:
assert(
    isinstance(btcusdt_1h_df, pd.DataFrame)
    and btcusdt_1h_df.shape == (24, 6)
    and not btcusdt_1h_df.isnull().any().any()
    and btcusdt_1h_df.iloc[:, 0:5].ge(0).all().all()
    )

Do the same for `BTC/USDC` pair:

In [10]:
btcusdc_df = get_data('BTCUSDC')
btcusdc_df = set_column_names(btcusdc_df)
btcusdc_df['Close_time'] = btcusdc_df.iloc[:, 6].apply(lambda t: datetime.fromtimestamp(t/1000))

btcusdc_1h_df = calculate_ohclv(btcusdc_df)
btcusdc_1h_df

,Open,High,Low,Close,Volume,Close_time
0,20549.65,20703.08,20371.75,20647.35,284.73521,2022-06-21 00:59:59.999
1,20648.06,20771.07,20482.83,20646.88,192.50513,2022-06-21 01:59:59.999
2,20644.78,20672.98,20331.36,20406.71,195.67023,2022-06-21 02:59:59.999
3,20402.27,20649.99,20331.48,20585.98,290.39210,2022-06-21 03:59:59.999
4,20586.63,20721.10,20449.57,20632.33,205.78459,2022-06-21 04:59:59.999
5,20640.33,20999.00,20594.58,20866.07,412.42477,2022-06-21 05:59:59.999
6,20873.25,21178.00,20873.25,21169.64,524.25894,2022-06-21 06:59:59.999
7,21166.58,21300.00,20913.40,21097.44,304.65113,2022-06-21 07:59:59.999
8,21097.20,21471.91,21023.64,21327.56,366.29201,2022-06-21 08:59:59.999
9,21342.76,21448.22,21144.24,21182.12,350.77543,2022-06-21 09:59:59.999


Join altogether:

In [11]:
btcusdt_1h_df['pair'] = 'BTC-USDT'
btcusdc_1h_df['pair'] = 'BTC-USDC'

candles_1h_df = pd.concat([btcusdt_1h_df, btcusdc_1h_df])

assert(
    isinstance(candles_1h_df, pd.DataFrame)
    and candles_1h_df.shape == (48, 7)
    and (candles_1h_df['pair'].unique() == ['BTC-USDT', 'BTC-USDC']).all()
)

Plot something interesting... :bulb:

Enrich dataset using Open FIGI API Interaction

In [12]:
def send_request(ticker: str) -> pd.DataFrame:
    api_url = f'https://www.openfigi.com/search/query?facetQuery=MARKET_SECTOR_DES:%22Curncy%22&num_rows=100&simpleSearchString={ticker}&start=0'

    response = httpx.get(api_url)
    json_response = response.json()
    
    return pd.DataFrame.from_dict(json_response['result'], orient='columns')

send_request('BTCUSDT')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)